# Gene set enrichment scoring
In this notebook, we show a small example of how to compute gene set enrichment scores. We simply construct all possible gene sets from 10 random genes and compute the gene set enrichment scores for one of the reference cells.

In [1]:
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from SEMITONES.enrichment_scoring import calculate_escores
from SEMITONES.enrichment_scoring import feature_set_values
from SEMITONES.support_funcs import pairwise_similarities
from SEMITONES.support_funcs import get_sets

## 1. Load the data

In [2]:
os.chdir("../nestorowa")

In [3]:
data = pd.read_hdf("nestorowa_processed.h5")  # counts
PC20 = np.loadtxt("nestorowa_20PC.txt", delimiter="\t")  # top 20 PCs
UMAP = np.loadtxt("nestorowa_UMAP.txt", delimiter="\t")  # 2D UMAP

In [4]:
with open("nestorowa_dd_rcells.txt", "r") as f:
    r_cells = [int(c.strip("\n")) for c in f.readlines()]
f.close()

## 2. Create the similarity matrix

In [5]:
g = 8.6e-4

In [6]:
S = pairwise_similarities(PC20,  # similarities in the 20 PC space
                          query=r_cells,  # only to the reference cells
                          metric="rbf",  # metric as in notebook 1
                          metric_params={"gamma": g})

## 3. Create gene sets

We pick 10 random genes from the data

In [7]:
genes = list(np.random.choice(data.columns, 10))
# get the gene indices for efficient computation on the np.matrix
genes_idx = [list(data.columns).index(gene) for gene in genes]

We can simply use itertools to get tuples of all possible gene sets. We turn it into a list so we can use it for mapping indices later.

In [8]:
genesets = list(itertools.combinations(genes_idx, r=2))

## 4. Create gene set expression matrices
To do enrichment scoring we have to combine the expression vectors for each gene set into one expression vector. Here, we use a simple interaction term. Options for "min", "max" and "median" expression are also implemented. For min, we take the minimum expression value out of the two genes in the set in each cell, for max we take the maximum expression value out of two genes in the set in each cell, and for median we take the median expression value at each index.

Because the gene sets are the same for each reference cell, we only construct one gsexp frame for the gene sets. However, if the gene sets are different per reference cell, you might wish to create a new expression frame for each computation

In [9]:
gsexp = feature_set_values(data.values, genesets, combtype="interaction")

Constructing expression set


Now, we can perform gene set enrichment scoring

In [10]:
escores = calculate_escores(gsexp, query=r_cells, S=S)

Start enrichment scoring using 32 CPUs
Creating process pool
Run enrichment scoring
Enrichment scoring complete


We can now map the gene set indices back to their gene names

In [11]:
genes, cells = list(data.columns), list(data.index)
genesets = [(genes[s[0]], genes[s[1]]) for s in genesets]
cellnames = [cells[i] for i in r_cells]

In [12]:
escores.index, escores.columns = genesets, cellnames

In [13]:
escores

,HSPC_117,HSPC_232,HSPC_336,LT-HSC_044,LT-HSC_068,LT-HSC_077,LT-HSC_124,Prog_029,Prog_182,Prog_210,Prog_369,Prog_487,Prog_488,Prog_729,Prog_734,Prog_736,Prog_775
"(Nicn1, Fah)",0.000290,0.000259,-0.001054,0.000333,-0.009869,0.000142,-0.000317,0.007473,0.001540,-0.000711,0.011442,-0.001084,-0.001650,-0.000210,0.004892,-0.005326,0.000059
"(Nicn1, Gabrd)",-0.000004,0.000042,0.000032,-0.000035,-0.005776,0.000071,0.000093,0.000626,-0.000012,-0.001675,-0.000294,-0.004215,-0.003460,-0.000835,-0.003369,-0.004687,0.000011
"(Nicn1, Col9a3)",0.000232,0.000240,0.000095,0.000306,-0.002656,0.000120,-0.000241,0.016081,0.001787,-0.003758,0.019071,0.001090,-0.004074,-0.003237,0.009255,-0.000640,0.000069
"(Nicn1, Mpped2)",0.000119,0.000082,0.000035,0.000094,-0.002734,0.000023,-0.001267,-0.002632,-0.000629,0.002991,-0.002719,-0.002143,0.001843,0.000152,0.000038,-0.001109,0.000025
"(Nicn1, Tcam1)",0.000082,-0.000048,0.000037,-0.000045,0.002746,0.000065,0.000645,0.000241,0.000364,0.000834,-0.002099,-0.000065,0.000582,0.000460,-0.001104,0.002567,0.000035
"(Nicn1, Gm9776)",0.000549,0.000087,0.000045,0.000082,-0.009028,0.000068,-0.000160,-0.002839,-0.002468,0.001399,-0.004602,-0.000286,0.001226,-0.000596,-0.008859,-0.007625,0.000009
"(Nicn1, Gm3696)",0.000102,0.000078,0.000041,0.000037,-0.003681,0.000055,-0.000626,0.000227,-0.001100,0.001551,0.000090,-0.002802,-0.001763,-0.000360,-0.001532,-0.003919,0.000016
"(Nicn1, Ms4a4c)",-0.002484,0.000013,-0.001321,0.000027,-0.004433,-0.001210,-0.001942,0.000933,0.001001,0.000644,0.002644,0.012488,0.005220,0.000344,0.000725,-0.001704,-0.000002
"(Nicn1, Il17re)",0.000209,0.000094,-0.000745,0.000062,-0.003252,0.000023,-0.000521,0.001155,-0.000669,-0.001532,0.002147,-0.001603,0.000464,-0.000272,-0.000565,-0.002043,0.000018
"(Fah, Gabrd)",0.000167,0.000243,-0.001539,-0.000043,-0.009372,0.000166,-0.000308,0.004815,0.001073,-0.000842,0.006371,0.002035,-0.003421,-0.000627,-0.003493,-0.005606,0.000051


## 5. Notes

We also supply a function called get_sets() where you can feed a dictionary of {cell_idx: [gene_idxs]} and get all possible gene sets per cell back. In this case, you should construct a gsexp frame for each cell idividually and do enrichment scoring on that frame, as shown below for two reference cells.

First, we create the gene sets using the function described above

In [14]:
genesub = list(np.random.choice(data.columns, 10))
# get the gene indices for efficient computation on the np.matrix
geneidx = [list(data.columns).index(gene) for gene in genesub]
geneidx = {r_cells[0]: geneidx,
           r_cells[1]: geneidx}
genesets = get_sets(geneidx, n=2, get_list=True)

Now, we subset our similarity matrix for the 2 cells we are using

In [15]:
escores = []
for r, sets in genesets.items():
    gsexp = feature_set_values(data.values, sets, combtype="interaction")
    r_indx = r_cells.index(r)  # map r to index
    Sr = S[:, r_indx][:, np.newaxis]  # subset S for only our reference cell
    e = calculate_escores(gsexp, query=[r], S=Sr)  # calculate scores
    cell = cells[r]  # map to cell name
    sets = [(genes[s[0]], genes[s[1]]) for s in sets] # map to gene names
    e.index, e.columns = sets, [cell]  # set names
    escores.append(e)

Constructing expression set
Start enrichment scoring using 32 CPUs
Creating process pool
Run enrichment scoring
Enrichment scoring complete
Constructing expression set
Start enrichment scoring using 32 CPUs
Creating process pool
Run enrichment scoring
Enrichment scoring complete


In [16]:
escores[0], escores[1]

(                           HSPC_117
 (Gm4890, Rpgrip1l)         0.000229
 (Gm4890, Rnf38)            0.000176
 (Gm4890, Zfp786)           0.000042
 (Gm4890, Alox5)            0.000569
 (Gm4890, Gm42141)          0.000088
 (Gm4890, Hs3st3b1)        -0.000116
 (Gm4890, Prrt3)            0.000073
 (Gm4890, Prdm9)           -0.000053
 (Gm4890, 1700101I11Rik)    0.000297
 (Rpgrip1l, Rnf38)          0.000253
 (Rpgrip1l, Zfp786)         0.000288
 (Rpgrip1l, Alox5)         -0.000078
 (Rpgrip1l, Gm42141)        0.000231
 (Rpgrip1l, Hs3st3b1)       0.000052
 (Rpgrip1l, Prrt3)          0.000008
 (Rpgrip1l, Prdm9)          0.000307
 (Rpgrip1l, 1700101I11Rik)  0.000045
 (Rnf38, Zfp786)            0.000220
 (Rnf38, Alox5)             0.000204
 (Rnf38, Gm42141)           0.000126
 (Rnf38, Hs3st3b1)         -0.000019
 (Rnf38, Prrt3)            -0.000189
 (Rnf38, Prdm9)            -0.000722
 (Rnf38, 1700101I11Rik)     0.000211
 (Zfp786, Alox5)            0.000065
 (Zfp786, Gm42141)          0.000254
 